<a href="https://www.kaggle.com/code/kelixirr/deepfood-food-vision-big-cnn?scriptVersionId=144281037" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf

In [ ]:
#importing helper function
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
import tensorflow_datasets as tfds

datasets_list = tfds.list_builders()

target_dataset = "food101"
print(f"'{target_dataset}' in TensorFlow Datasets: {target_dataset in datasets_list}")

In [ ]:
# loading the data 
(train_data, test_data), ds_info = tfds.load(name = 'food101',
                                            split = ['train', 'validation'],
                                            shuffle_files = True,
                                            as_supervised = True,
                                            with_info = True)

In [ ]:
ds_info.features

In [ ]:
class_names = ds_info.features['label'].names
class_names[:10]

In [ ]:
train_one_sample = train_data.take(1)

In [ ]:
train_one_sample

In [ ]:
for image, label in train_one_sample:
    print(f"""
    Image shape: {image.shape}
    Image dtype: {image.dtype}
    Target class from Food101 (tensor form): {label}
    Class name (str form): {class_names[label.numpy()]}
        """)

In [ ]:
image

In [ ]:
# min and max values
tf.reduce_min(image), tf.reduce_max(image)

In [ ]:
import matplotlib.pyplot as plt 
plt.imshow(image)
plt.title(class_names[label])
plt.axis()

## Create Preprocessing Functions Of Our Data

In [ ]:
image.shape, image.dtype

In [ ]:
# prorcessing image

def preprocess_img(image, label, img_shape = 224):
    
    image = tf.image.resize(image, [img_shape, img_shape]) # reshaping image
    return tf.cast(image, tf.float32), label

In [ ]:
# checking our function 
preprocessed_img = preprocess_img(image, label)[0]
preprocessed_img

In [ ]:
plt.imshow(preprocessed_img/255.)
plt.title(class_names[label])
plt.axis(False);

In [ ]:
# map the preprocessing image function 
train_data = train_data.map(map_func = preprocess_img, num_parallel_calls = tf.data.AUTOTUNE)

# shuffle data 
train_data = train_data.shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

In [ ]:
# turning on mixed precision training for efficiency purpose
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy = 'mixed_float16')

In [ ]:
mixed_precision.global_policy()

## Feature Extraction Model

In [ ]:
from tensorflow.keras import layers

# create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# function model 
inputs = layers.Input(shape = input_shape, name = "input_layer")

x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name = 'pooling_layer')(x)
x = layers.Dense(len(class_names))(x)

outputs = layers.Activation("softmax", dtype = tf.float32, name = 'softmax_float32')(x)

model = tf.keras.Model(inputs, outputs)

# compile the model 
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
# checking for layers dtype policy
for layer in model.layers:
    print(layer.name, layer.dtype_policy, layer.name, layer.dtype, layer.trainable)

In [ ]:
for layer in model.layers[1].layers[:20]:
    print(layer.name, layer.dtype_policy, layer.name, layer.dtype, layer.trainable)

In [ ]:
# fit the model
history_feature_extraction = model.fit(train_data,
                                       epochs = 3,
                                       steps_per_epoch = len(train_data),
                                       validation_data = test_data,
                                       validation_steps = int(0.15 * len(test_data)))

In [ ]:
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

In [ ]:
save_dir = "Efficientnetb0_feature_extract_model_mixed_precision"
model.save(save_dir)

### Please note: the above error is happening due to some bug in new version tensorflow. 

In [ ]:
# Load model previously saved above
loaded_saved_model = tf.keras.models.load_model(save_dir)

In [ ]:
results_loaded_saved_model = loaded_saved_model.evaluate(test_data)
results_loaded_saved_model

In [ ]:
 #The loaded model's results should equal (or at least be very close) to the model's results prior to saving
import numpy as np
assert np.isclose(results_feature_extract_model, results_loaded_saved_model).all()

## Fine Tuning Our Model

In [ ]:
# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip 

In [ ]:
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

In [ ]:
# Load and evaluate downloaded GS model
loaded_gs_model = tf.keras.models.load_model("downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
# Get a summary of our downloaded model
loaded_gs_model.summary()

In [ ]:
# How does the loaded model perform?
results_loaded_gs_model = loaded_gs_model.evaluate(test_data)
results_loaded_gs_model

In [ ]:
# Are any of the layers in our model frozen?
for layer in loaded_gs_model.layers:
    layer.trainable = True # set all layers to trainable
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
for layer in loaded_gs_model.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
## creating callbacks for optimizing training time
from helper_functions import create_tensorboard_callback

#early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",
                                                  patience = 3) # if val loss decreases for 3 epochs in a row, stop training

# model checkpoint
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only = True,
                                                      monitor = 'val_loss')

#learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss",
                                                 factor = 0.2, # reduce lr by 5x
                                                 patience = 2,
                                                 verbose = 1,
                                                 min_lr = 1e-7)

In [ ]:
# Compile the model
loaded_gs_model.compile(loss="sparse_categorical_crossentropy", # for labels that are not one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate 
                        metrics=["accuracy"])

In [ ]:
history_fine_tuned_model = loaded_gs_model.fit(train_data,
                                               epochs = 100,
                                               steps_per_epoch = len(train_data),
                                               validation_data = test_data,
                                               validation_steps = int(0.15 * len(test_data)),
                                               callbacks = [create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"),
                                                            model_checkpoint, # save only the best model during training
                                                            early_stopping,
                                                            reduce_lr]) 

In [ ]:
# loaded_gs_model.save("07_efficientnetb0_fine_tuned_101_classes_mixed_precision")  # not saving due to bug in Tf 10. above

In [ ]:
# Downloading the fine tuned model from Google Drive
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip

In [ ]:
# Unzip fine-tuned model
!mkdir downloaded_fine_tuned_gs_model # create separate directory for fine-tuned model downloaded from Google Storage
!unzip 07_efficientnetb0_fine_tuned_101_classes_mixed_precision -d downloaded_fine_tuned_gs_model

In [ ]:
# Load in fine-tuned model from Google Storage and evaluate
loaded_fine_tuned_gs_model = tf.keras.models.load_model("downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision")

In [ ]:
# Get a model summary (same model architecture as above)
loaded_fine_tuned_gs_model.summary()

In [ ]:
results_downloaded_fine_tuned_gs_model = loaded_fine_tuned_gs_model.evaluate(test_data)
results_downloaded_fine_tuned_gs_model